In [1]:
import numpy as np
import pandas as pd
import glob
import json
import os
import sys
import urllib.request
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import datetime as dt
import pymysql

In [3]:
chromedriver_dir = 'D:\dev\driver\chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [51]:
driver.get("https://kr.investing.com/indices/us-30-historical-data")

In [52]:
openDate = driver.find_element_by_id('widgetFieldDateRange')
openDate.click()

startDate = driver.find_element_by_id('startDate')
startDate.clear()
startDate.send_keys('2013/01/01')

endDate = driver.find_element_by_id('endDate')
endDate.clear()
endDate.send_keys('2020/01/16')

applyBtn = driver.find_element_by_id('applyBtn')
applyBtn.click()

time.sleep(3) #로딩시간 감안

In [5]:
source = driver.page_source
soupData = BeautifulSoup(source,'html.parser')

dataTable = soupData.find('table',id='curr_table')

In [ ]:
tableCols = dataTable.find_all('th')

tableColList = np.array([])
for i in range(len(tableCols)) :
    print(tableCols[i].text)
    tableColList = np.append(tableColList,tableCols[i].text)

In [9]:
len(dataTable.find_all('tr'))

1774

In [11]:
for i in range(len(dataTable.find_all('tr')[0].find_all('th'))) :
    print(dataTable.find_all('tr')[0].find_all('th')[i].text)

날짜
현재가
오픈
고가
저가
거래량
변동 %


In [12]:
DJ_df = pd.DataFrame([])

In [ ]:
# 단숨에 넣는것이 아닌 한줄씩 넣어야하는게 맞느듯?
DJdate = np.array([])
DJclose = np.array([])
DJopen = np.array([])
DJhigh = np.array([])
DJlow = np.array([])
DJvolume = np.array([])

for i in range(1, len(dataTable.find_all('tr'))+1) :
    



In [34]:
dataTable.find_all('tr')[1].find_all('td')[1].text

'29,297.64'

In [31]:
len(dataTable.find_all('tr'))

1774

In [30]:
DJ_df = pd.DataFrame([])
tmp_df = pd.DataFrame([])
tmp_df = tmp_df.append({'date':'','close':'','open':'','high':'','low':'','volume':''}, ignore_index=True)

In [ ]:
# DataFrame에 넣는 코드

In [44]:
for i in range(1,len(dataTable.find_all('tr'))) :
    tmpDateStr = ''
    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' )
    tmp_df.at[0,'date'] = tmpDate
    
    tmpClose = 0
    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))
    tmp_df.at[0,'close'] = tmpClose
    
    tmpOpen = 0
    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))
    tmp_df.at[0,'open'] = tmpOpen
    
    tmpHigh = 0
    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))
    tmp_df.at[0,'high'] = tmpHigh
    
    tmpLow = 0
    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))
    tmp_df.at[0,'low'] = tmpLow
    
    tmpVolumeStr = ''
    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")
    
    if tmpVolumeStr[-1] == 'M' :
        tmpVolume = float(tmpVolumeStr.split('M')[0])*1000000
    elif tmpVolumeStr[-1] == 'k' :
        tmpVolume = float(tmpVolumeStr.split('k')[0])*1000
    else : 
        tmpVolume = float(tmpVolumeStr)
    tmp_df.at[0,'volume'] = tmpVolume
        
    DJ_df = DJ_df.append(tmp_df,ignore_index = True)
    
    for j in range(len(tmp_df.columns)) :
        tmp_df.iat[0,j] = ''
    


In [45]:
DJ_df

,close,date,high,low,open,volume
0,29297.6,2020-01-16 00:00:00,29300.3,29132,29132,2.5211e+08
1,29030.2,2020-01-15 00:00:00,29127.6,28897.3,28901.8,2.6343e+08
2,28939.7,2020-01-14 00:00:00,29054.2,28872.3,28895.5,2.9604e+08
3,28907,2020-01-13 00:00:00,28909.9,28819.4,28869,2.6302e+08
4,28823.8,2020-01-10 00:00:00,29009.1,28789.1,28977.5,2.3917e+08
...,...,...,...,...,...,...
1768,13328.9,2013-01-08 00:00:00,13377.4,13293.1,13377.4,1.2957e+08
1769,13384.3,2013-01-07 00:00:00,13436.1,13343.3,13436.1,1.1312e+08
1770,13435.2,2013-01-04 00:00:00,13447.1,13376.2,13391,1.0759e+08
1771,13391.4,2013-01-03 00:00:00,13430.6,13358.3,13413,1.2963e+08


In [ ]:
# db에 최초 실행시 박아넣는 코드

In [64]:
conn = pymysql.connect(host='localhost', port=3306,
                       user='acorn12', passwd='acorn12', db='acorn', charset='utf8')

try:
    with conn.cursor() as cursor:
                
        for i in range(1,len(dataTable.find_all('tr'))) :
            tmpDateStr = ''
            tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
            tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

            tmpClose = 0
            tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

            tmpOpen = 0
            tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

            tmpHigh = 0
            tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

            tmpLow = 0
            tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

            tmpVolumeStr = ''
            tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

            if tmpVolumeStr[-1] == 'M' :
                tmpVolume = float(tmpVolumeStr.split('M')[0])*1000000
            elif tmpVolumeStr[-1] == 'k' :
                tmpVolume = float(tmpVolumeStr.split('k')[0])*1000
            else : 
                tmpVolume = float(tmpVolumeStr)


            sql = 'INSERT INTO dowjones(dj_date, dj_close, dj_open, dj_high, dj_low, dj_volume) '
            sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

            cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

        conn.commit()
        
finally:
    conn.close()


    

IntegrityError: (1062, "Duplicate entry '2020-01-16' for key 'PRIMARY'")

In [ ]:
# db 최근날짜와 비교해서 확인하는 코드

In [ ]:
# 최근 가장 최근날짜나 2013-01-01 반납코드

In [67]:
conn = pymysql.connect(host='localhost', port=3306,
                       user='acorn12', passwd='acorn12', db='acorn', charset='utf8')

try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(dj_date),"2013-01-01") FROM dowjones '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentDate = pd.to_datetime(lastDate).date()
        recentDate += datetime.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        print(recentDate,today)
        
        conn.commit()
        

finally:
    conn.close()

2020-01-17 2020-01-20


In [62]:
print(type(pd.to_datetime('2020-01-20').date()))

<class 'datetime.date'>


In [ ]:
# 데이터 최근부터(없으면 13-01-01부터 확인하고 긁어오는 코드)


In [6]:
# dowjones 데이터 입력

# db 커넥션 연결
# conn = pymysql.connect(host='localhost', port=3306,
#                        user='acorn12', passwd='acorn12', db='acorn', charset='utf8')

conn = pymysql.connect(host='localhost', port=3306,
                       user='flo', passwd='flo', db='db', charset='utf8')


try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(dj_date),"2013-01-01") FROM dowjones '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += dt.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        
        if recentlyDate < today :
            # 셀레니움 연결
            chromedriver_dir = 'D:\dev\driver\chromedriver.exe'
            driver = webdriver.Chrome(chromedriver_dir)

            # 대상 주소
            driver.get("https://kr.investing.com/indices/us-30-historical-data")



            # 날짜는 수정하는걸 확인해서 만드는거
            openDate = driver.find_element_by_id('widgetFieldDateRange')
            openDate.click()

            startDate = driver.find_element_by_id('startDate')
            startDate.clear()
            startDate.send_keys(recentlyDate.strftime('%Y/%m/%d'))

            endDate = driver.find_element_by_id('endDate')
            endDate.clear()
            endDate.send_keys(today.strftime('%Y/%m/%d'))

            applyBtn = driver.find_element_by_id('applyBtn')
            applyBtn.click()

            time.sleep(5) #로딩시간 감안
            # 직접 개체를 받아와서 

            source = driver.page_source
            soupData = BeautifulSoup(source,'html.parser')

            dataTable = soupData.find('table',id='curr_table')

            # 해당 테이블로 데이터를 집어넣는 것
            if (len(dataTable.find_all('tr')) > 1) & (len(dataTable.find_all('tr')[1].find_all('td')) != 1) :
                for i in range(1,len(dataTable.find_all('tr'))) :
                    tmpDateStr = ''
                    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
                    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

                    tmpClose = 0
                    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

                    tmpOpen = 0
                    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

                    tmpHigh = 0
                    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

                    tmpLow = 0
                    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

                    tmpVolumeStr = ''
                    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

                    if tmpVolumeStr[-1] in ['m', 'M'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000000
                    elif tmpVolumeStr[-1] in ['k','K'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000
                    else : 
                        tmpVolume = float(tmpVolumeStr)


                    sql = 'INSERT INTO dowjones(dj_date, dj_close, dj_open, dj_high, dj_low, dj_volume) '
                    sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                    cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                conn.commit()
        

finally:
    conn.close()
    driver.close()

In [9]:
# gold 데이터 입력

# db 커넥션 연결
# conn = pymysql.connect(host='localhost', port=3306,
#                        user='acorn12', passwd='acorn12', db='acorn', charset='utf8')

conn = pymysql.connect(host='localhost', port=3306,
                       user='flo', passwd='flo', db='db', charset='utf8')



try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(gold_date),"2013-01-01") FROM gold '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += dt.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        if recentlyDate < today :
            # 셀레니움 연결
            chromedriver_dir = 'D:\dev\driver\chromedriver.exe'
            driver = webdriver.Chrome(chromedriver_dir)

            # 대상 주소
            driver.get("https://kr.investing.com/commodities/gold-historical-data")

            # 날짜는 수정하는걸 확인해서 만드는거
            openDate = driver.find_element_by_id('widgetFieldDateRange')
            openDate.click()

            startDate = driver.find_element_by_id('startDate')
            startDate.clear()
            startDate.send_keys(recentlyDate.strftime('%Y/%m/%d'))

            endDate = driver.find_element_by_id('endDate')
            endDate.clear()
            endDate.send_keys(today.strftime('%Y/%m/%d'))

            applyBtn = driver.find_element_by_id('applyBtn')
            applyBtn.click()

            time.sleep(5) #로딩시간 감안
            # 직접 개체를 받아와서 

            source = driver.page_source
            soupData = BeautifulSoup(source,'html.parser')

            dataTable = soupData.find('table',id='curr_table')

            # 해당 테이블로 데이터를 집어넣는 것
            if (len(dataTable.find_all('tr')) > 1) & (len(dataTable.find_all('tr')[1].find_all('td')) != 1) :
                for i in range(1,len(dataTable.find_all('tr'))) :
                    tmpDateStr = ''
                    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
                    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

                    tmpClose = 0
                    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

                    tmpOpen = 0
                    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

                    tmpHigh = 0
                    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

                    tmpLow = 0
                    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

                    tmpVolumeStr = ''
                    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

                    if tmpVolumeStr[-1] in ['m', 'M'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000000
                    elif tmpVolumeStr[-1] in ['k','K'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000
                    elif tmpVolumeStr == '-' :
                        tmpVolume = 0
                    else : 
                        tmpVolume = float(tmpVolumeStr)


                    sql = 'INSERT INTO gold(gold_date, gold_close, gold_open, gold_high, gold_low, gold_volume) '
                    sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                    cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                conn.commit()
        

finally:
    driver.close()
    conn.close()
   

In [10]:
# bitcoin 데이터 입력

# db 커넥션 연결
# conn = pymysql.connect(host='localhost', port=3306,
#                        user='acorn12', passwd='acorn12', db='acorn', charset='utf8')

conn = pymysql.connect(host='localhost', port=3306,
                       user='flo', passwd='flo', db='db', charset='utf8')



try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(bit_date),"2013-01-01") FROM bitcoin '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += dt.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        if recentlyDate < today :
            # 셀레니움 연결
            chromedriver_dir = 'D:\dev\driver\chromedriver.exe'
            driver = webdriver.Chrome(chromedriver_dir)

            # 대상 주소
            driver.get("https://kr.investing.com/crypto/bitcoin/btc-usd-historical-data")

            # 날짜는 수정하는걸 확인해서 만드는거
            openDate = driver.find_element_by_id('widgetFieldDateRange')
            openDate.click()

            startDate = driver.find_element_by_id('startDate')
            startDate.clear()
            startDate.send_keys(recentlyDate.strftime('%Y/%m/%d'))

            endDate = driver.find_element_by_id('endDate')
            endDate.clear()
            endDate.send_keys(today.strftime('%Y/%m/%d'))

            applyBtn = driver.find_element_by_id('applyBtn')
            applyBtn.click()

            time.sleep(5) #로딩시간 감안
            # 직접 개체를 받아와서 

            source = driver.page_source
            soupData = BeautifulSoup(source,'html.parser')

            dataTable = soupData.find('table',id='curr_table')

            # 해당 테이블로 데이터를 집어넣는 것
            if (len(dataTable.find_all('tr')) > 1) & (len(dataTable.find_all('tr')[1].find_all('td')) != 1) :
                for i in range(1,len(dataTable.find_all('tr'))) :
                    tmpDateStr = ''
                    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
                    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

                    tmpClose = 0
                    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

                    tmpOpen = 0
                    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

                    tmpHigh = 0
                    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

                    tmpLow = 0
                    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

                    tmpVolumeStr = ''
                    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

                    if tmpVolumeStr[-1] in ['m', 'M'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000000
                    elif tmpVolumeStr[-1] in ['k','K'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000
                    elif tmpVolumeStr == '-' :
                        tmpVolume = 0
                    else : 
                        tmpVolume = float(tmpVolumeStr)


                    sql = 'INSERT INTO bitcoin(bit_date, bit_close, bit_open, bit_high, bit_low, bit_volume) '
                    sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                    cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                conn.commit()
        

finally:
    driver.close()
    conn.close()

In [94]:
# ether 데이터 입력 - investing 쪽 데이터가 많이 없기때문에 coinmarketcap으로 전환함

# db 커넥션 연결
conn = pymysql.connect(host='localhost', port=3306,
                       user='acorn12', passwd='acorn12', db='acorn', charset='utf8')



try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(eth_date),"2013-01-01") FROM ether '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += dt.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        if recentlyDate < today :
            # 셀레니움 연결
            chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
            driver = webdriver.Chrome(chromedriver_dir)

            # 대상 주소
            driver.get("https://kr.investing.com/crypto/ethereum/eth-usd-historical-data")

            # 날짜는 수정하는걸 확인해서 만드는거
            openDate = driver.find_element_by_id('widgetFieldDateRange')
            openDate.click()

            startDate = driver.find_element_by_id('startDate')
            startDate.clear()
            startDate.send_keys(recentlyDate.strftime('%Y/%m/%d'))

            endDate = driver.find_element_by_id('endDate')
            endDate.clear()
            endDate.send_keys(today.strftime('%Y/%m/%d'))

            applyBtn = driver.find_element_by_id('applyBtn')
            applyBtn.click()

            time.sleep(5) #로딩시간 감안
            # 직접 개체를 받아와서 

            source = driver.page_source
            soupData = BeautifulSoup(source,'html.parser')

            dataTable = soupData.find('table',id='curr_table')

            # 해당 테이블로 데이터를 집어넣는 것
            if (len(dataTable.find_all('tr')) > 1) & (len(dataTable.find_all('tr')[1].find_all('td')) != 1) :
                for i in range(1,len(dataTable.find_all('tr'))) :
                    tmpDateStr = ''
                    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
                    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

                    tmpClose = 0
                    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

                    tmpOpen = 0
                    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

                    tmpHigh = 0
                    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

                    tmpLow = 0
                    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

                    tmpVolumeStr = ''
                    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

                    if tmpVolumeStr[-1] in ['m', 'M'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000000
                    elif tmpVolumeStr[-1] in ['k','K'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000
                    elif tmpVolumeStr == '-' :
                        tmpVolume = 0
                    else : 
                        tmpVolume = float(tmpVolumeStr)


                    sql = 'INSERT INTO ether(eth_date, eth_close, eth_open, eth_high, eth_low, eth_volume) '
                    sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                    cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                conn.commit()
        

finally:
    conn.close()

In [ ]:
# ether 데이터 입력 - coinmarketcap 쪽 데이터 크롤링

# db 커넥션 연결
conn = pymysql.connect(host='localhost', port=3306,
                       user='acorn12', passwd='acorn12', db='acorn', charset='utf8')



try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(eth_date),"2013-01-01") FROM ether '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += datetime.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        if recentlyDate < today :
            # 셀레니움 연결
            chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
            driver = webdriver.Chrome(chromedriver_dir)

            # 대상 주소
            
            url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start='
            url += recentlyDate.strftime('%Y%m%d') + '&end=' + today.strftime('%Y%m%d')
            
            driver.get(url)
            
            time.sleep(5) #로딩시간 감안
            # 직접 개체를 받아와서 

            source = driver.page_source
            soupData = BeautifulSoup(source,'html.parser')

            dataTable = soupData.find_all('tr',class_='cmc-table-row')

            # 해당 테이블로 데이터를 집어넣는 것
            if (len(dataTable) > 1) & (len(dataTable[1].find_all('td')) != 1) :
                for i in range(1,len(dataTable)) :
                    tmpDateStr = ''
                    tmpDateStr = dataTable.find_all('tr')[i].find_all('td')[0].text
                    tmpDate = dt.datetime.strptime(tmpDateStr, '%Y년 %m월 %d일' ).date()

                    tmpClose = 0
                    tmpClose = float(dataTable.find_all('tr')[i].find_all('td')[1].text.replace(',',""))

                    tmpOpen = 0
                    tmpOpen = float(dataTable.find_all('tr')[i].find_all('td')[2].text.replace(',',""))

                    tmpHigh = 0
                    tmpHigh = float(dataTable.find_all('tr')[i].find_all('td')[3].text.replace(',',""))

                    tmpLow = 0
                    tmpLow = float(dataTable.find_all('tr')[i].find_all('td')[4].text.replace(',',""))

                    tmpVolumeStr = ''
                    tmpVolumeStr = dataTable.find_all('tr')[i].find_all('td')[5].text.replace(',',"")

                    if tmpVolumeStr[-1] in ['m', 'M'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000000
                    elif tmpVolumeStr[-1] in ['k','K'] :
                        tmpVolume = float(tmpVolumeStr[:-1])*1000
                    elif tmpVolumeStr == '-' :
                        tmpVolume = 0
                    else : 
                        tmpVolume = float(tmpVolumeStr)


                    sql = 'INSERT INTO ether(eth_date, eth_close, eth_open, eth_high, eth_low, eth_volume) '
                    sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                    cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                conn.commit()
        

finally:
    conn.close()

In [8]:
# ether 데이터 입력 - coinmarketcap 쪽 데이터 크롤링
# 동적 크롤링이 필요없기때문에 selenium을 버리고한다
# 일반 데이터 크롤링 방법 - 셀레니움보다 훨씬 빠르다


# db 커넥션 연결
conn = pymysql.connect(host='localhost', port=3306,
                       user='acorn12', passwd='acorn12', db='acorn', charset='utf8')



try:
    with conn.cursor() as cursor:
        sql = 'SELECT IFNULL(MAX(eth_date),"2013-01-01") FROM ether '
        
        cursor.execute(sql)
        conn.commit()
        
        rows = cursor.fetchone()
        lastDate = rows[0]
        
        # db에 기록된 가장 최근값과 오늘의 날짜 반납( 이 기준으로 db에서 가져옴)
        
        recentlyDate = pd.to_datetime(lastDate).date()
        recentlyDate += dt.timedelta(days=1) # 하루 추가하기(다음날로 확인하는 거)
        today = dt.datetime.now().date()
        #print(recentDate,today)
        
        if recentlyDate < today :
            url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start='
            url += recentlyDate.strftime('%Y%m%d') + '&end=' + today.strftime('%Y%m%d')
            
            # 대상 주소
            with urllib.request.urlopen(url) as response :
                html = response.read()
                soupData = BeautifulSoup(html,'html.parser')
            

                dataTable = soupData.find_all('tr',class_='cmc-table-row')

                # 해당 테이블로 데이터를 집어넣는 것
                if (len(dataTable) > 0) & (len(dataTable[1].find_all('td')) != 1) :
                    for i in range(1,len(dataTable)) :
                        tmpDateStr = ''
                        tmpDateStr = dataTable[i].find_all('td')[0].text
                        tmpDate = pd.to_datetime(tmpDateStr).date()

                        tmpClose = 0
                        tmpClose = float(dataTable[i].find_all('td')[4].text.replace(',',""))

                        tmpOpen = 0
                        tmpOpen = float(dataTable[i].find_all('td')[1].text.replace(',',""))

                        tmpHigh = 0
                        tmpHigh = float(dataTable[i].find_all('td')[2].text.replace(',',""))

                        tmpLow = 0
                        tmpLow = float(dataTable[i].find_all('td')[3].text.replace(',',""))

                        tmpVolumeStr = ''
                        tmpVolumeStr = dataTable[i].find_all('td')[5].text.replace(',',"")

                        if tmpVolumeStr[-1] in ['m', 'M'] :
                            tmpVolume = float(tmpVolumeStr[:-1])*1000000
                        elif tmpVolumeStr[-1] in ['k','K'] :
                            tmpVolume = float(tmpVolumeStr[:-1])*1000
                        elif tmpVolumeStr == '-' :
                            tmpVolume = 0
                        else : 
                            tmpVolume = float(tmpVolumeStr)


                        sql = 'INSERT INTO ether(eth_date, eth_close, eth_open, eth_high, eth_low, eth_volume) '
                        sql += 'VALUES(%s,%s,%s,%s,%s,%s) '

                        cursor.execute(sql,(tmpDate,tmpClose,tmpOpen,tmpHigh,tmpLow,tmpVolume) )

                    conn.commit()
        

finally:
    conn.close()

In [35]:
dataTable

[<tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 20, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">166.90</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">169.11</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">162.33</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">167.12</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">11,144,529,787</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">18,276,600,992</div></td></tr>,
 <tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 19, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">175.47</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">177.25</di

In [98]:
# 셀레니움 연결
chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

# 대상 주소

url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start='
url += recentlyDate.strftime('%Y%m%d') + '&end=' + today.strftime('%Y%m%d')

driver.get(url)

time.sleep(5) #로딩시간 감안
# 직접 개체를 받아와서 

source = driver.page_source
soupData = BeautifulSoup(source,'html.parser')



In [103]:
divTable = soupData.find('div',class_='cmc-table__table-wrapper-outer')
dataTable = divTable.find_all('tbody')


In [108]:
trTable = soupData.find_all('tr',class_='cmc-table-row')
trTable

[<tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 19, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">175.47</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">177.25</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">163.68</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">166.97</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">13,713,374,132</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">18,257,853,532</div></td></tr>,
 <tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 18, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">170.75</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">178.53</di

In [118]:
trTable[0].find_all('td')[2].text

'177.25'

In [106]:
divTable

<div class="cmc-table__table-wrapper-outer cmc-table__table-wrapper-outer--fixed" style="display: none;"><div><table><thead><tr><th class="cmc-table__cell cmc-table__header cmc-table__cell--sticky cmc-table__cell--left">Date</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">Open*</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">High</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">Low</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">Close**</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">Volume</th><th class="cmc-table__cell cmc-table__header cmc-table__cell--right">Market Cap</th></tr></thead></table></div></div>

In [101]:
len(dataTable.find_all('tr'))

1

In [17]:
recentlyDate = pd.to_datetime('2019-12-01').date()

In [18]:
recentlyDate

datetime.date(2019, 12, 1)

In [5]:
#beatifulSoup

today = dt.datetime.now().date()


In [19]:
url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start='
url += recentlyDate.strftime('%Y%m%d') + '&end=' + today.strftime('%Y%m%d')

In [20]:
url

'https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20191201&end=20200121'

In [21]:
with urllib.request.urlopen(url) as response :
    html = response.read()
    soupData = BeautifulSoup(html,'html.parser')

In [22]:
trTable = soupData.find_all('tr',class_='cmc-table-row')


In [23]:
trTable

[<tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 20, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">166.90</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">169.11</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">162.33</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">167.12</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">11,144,529,787</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">18,276,600,992</div></td></tr>,
 <tr class="cmc-table-row" style="display:table-row"><td class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--left"><div class="">Jan 19, 2020</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">175.47</div></td><td class="cmc-table__cell cmc-table__cell--right"><div class="">177.25</di

In [29]:
trTable[0].find_all('td')[0].text

'Jan 20, 2020'